In [ ]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [ ]:
%run ./Keys.ipynb

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search_tweets, q=search_words,lang="en", since=date_since).items(12452)

In [ ]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

In [ ]:
print(f"new tweets retrieved: {len(tweets_copy)}")

In [ ]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.to_csv('covid19_tweets.csv')

In [ ]:
tweets_df.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tweets_df['source'].unique

In [ ]:
tweets_df['source'].value_counts()

### Tweetlerde Test Analizi

In [ ]:
pip install neattext

In [ ]:
import neattext.functions as nfx

In [ ]:
dir(nfx)

In [ ]:
tweets_df['text']

### Noise
+Twittlerdeki bahsetmeleri, hashtagleri, url ve emojileri kaldırma

In [ ]:
tweets_df['text'].apply(nfx.extract_hashtags)

In [ ]:
tweets_df['extract_hashtags'] = tweets_df['text'].apply(nfx.extract_hashtags)

In [ ]:
tweets_df[['extract_hashtags','hashtags']]

In [ ]:
#Textlerden hashtagleri kaldirma
tweets_df['clean_tweet'] = tweets_df['text'].apply(nfx.remove_hashtags)

In [ ]:
tweets_df[['text', 'clean_tweet']]

In [ ]:
tweets_df['clean_tweet'] = tweets_df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))

In [ ]:
tweets_df[['text', 'clean_tweet']]

In [ ]:
#Textlerden boşlukları kaldirma
tweets_df['clean_tweet'] = tweets_df['clean_tweet'].apply(nfx.remove_multiple_spaces)

In [ ]:
#Textlerden url kaldirma
tweets_df['clean_tweet'] = tweets_df['clean_tweet'].apply(nfx.remove_urls)

In [ ]:
#Textlerden noktalama işaretleri kaldirma
tweets_df['clean_tweet'] = tweets_df['clean_tweet'].apply(nfx.remove_puncts)

In [ ]:
tweets_df[['text', 'clean_tweet']]